In [1]:
# Import packages
import numpy as np
import tensorflow as tf
tf.__version__

from tensorflow.keras.datasets import cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPool2D, BatchNormalization, Input, Lambda
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.backend import repeat_elements, expand_dims, resize_images
# from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow.keras.backend as K

from sklearn.metrics import log_loss
import time
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
import cv2



In [2]:
(X_train, Y_train), (X_test, Y_test) = cifar100.load_data()
nb_train_samples = len(X_train)
nb_test_samples = len(X_test)
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [3]:
X_train = np.array([cv2.resize(img, (96,96)) for img in X_train[:nb_train_samples,:,:,:]])
X_test = np.array([cv2.resize(img, (96,96)) for img in X_test[:nb_test_samples,:,:,:]])

## Define custom loss functions and metrics

In [4]:
def superclass1_loss(y_true,y_pred):
    # This is the cross entropy loss at the superclass level (20 superclasses)
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    part1 = [[4, 30, 55, 72, 95],[1, 32, 67, 73, 91],[54, 62, 70, 82, 92],
             [9, 10, 16, 28, 61],[0, 51, 53, 57, 83],[22, 39, 40, 86, 87],
             [5, 20, 25, 84, 94],[6, 7, 14, 18, 24], [3, 42, 43, 88, 97],
             [12, 17, 37, 68, 76], [23, 33, 49, 60, 71], [15, 19, 21, 31, 38],
             [34, 63, 64, 66, 75], [26, 45, 77, 79, 99], [2, 11, 35, 46, 98],
             [27, 29, 44, 78, 93], [36, 50, 65, 74, 80], [47, 52, 56, 59, 96],
             [8, 13, 48, 58, 90], [41, 69, 81, 85, 89]]
    loss1=0
    for sublist in part1:
        ytt_temp = K.sum(tf.gather(y_true,sublist,axis=1),axis=-1)
        ypt_temp = K.sum(tf.gather(y_pred,sublist,axis=1),axis=-1)
        loss1+=ytt_temp*(-1)*K.log(ypt_temp)
    return loss1

In [5]:
def superclass2_loss(y_true,y_pred):
    # This is the cross entropy loss at the level 1 up from superclass (8 choices)
    # 4 animals (human, mammal, water, other)
    # plant, landscape, and 2 objects (vehicle, other)
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    part2 = [[4, 30, 55, 72, 95, 1, 32, 67, 73, 91],
             [54, 62, 70, 82, 92, 0, 51, 53, 57, 83, 47, 52, 56, 59, 96],
             [6, 7, 14, 18, 24, 26, 45, 77, 79, 99, 27, 29, 44, 78, 93],
             [3, 42, 43, 88, 97, 15, 19, 21, 31, 38, 34, 63, 64, 66, 75, 36, 50, 65, 74, 80],
             [2, 11, 35, 46, 98],
             [23, 33, 49, 60, 71, 12, 17, 37, 68, 76],
             [9, 10, 16, 28, 61, 22, 39, 40, 86, 87, 5, 20, 25, 84, 94],
             [8, 13, 48, 58, 90, 41, 69, 81, 85, 89]]

    loss2=0
    for sublist in part2:
        ytt_temp = K.sum(tf.gather(y_true,sublist,axis=1),axis=-1)
        ypt_temp = K.sum(tf.gather(y_pred,sublist,axis=1),axis=-1)
        loss2+=ytt_temp*(-1)*K.log(ypt_temp)
    return loss2

In [6]:
def superclass3_loss(y_true,y_pred):
    # This is the cross entropy loss at the level 2 up from superclass (4 choices)
    # animal, plant, landscape, object (manmade)
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    part3 = [[4, 30, 55, 72, 95, 1, 32, 67, 73, 91, 6, 7, 14, 18, 24, 26, 45, 77, 79,
              99, 27, 29, 44, 78, 93, 3, 42, 43, 88, 97, 15, 19, 21, 31, 38, 34, 63, 64,
              66, 75, 36, 50, 65, 74, 80, 2, 11, 35, 46, 98],
             [54, 62, 70, 82, 92, 0, 51, 53, 57, 83, 47, 52, 56, 59, 96],
             [23, 33, 49, 60, 71, 12, 17, 37, 68, 76],
             [9, 10, 16, 28, 61, 22, 39, 40, 86, 87, 5, 20, 25, 84, 94, 8, 13, 48, 58,
              90, 41, 69, 81, 85, 89]]
    loss3=0
    for sublist in part3:
        ytt_temp = K.sum(tf.gather(y_true,sublist,axis=1),axis=-1)
        ypt_temp = K.sum(tf.gather(y_pred,sublist,axis=1),axis=-1)
        loss3+=ytt_temp*(-1)*K.log(ypt_temp)
    return loss3

In [7]:
def struct1_loss(y_true,y_pred):
    # This is an equal blend of superclass_loss and "normal" (class-level) loss
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    loss0 = K.sum(y_true*(-1)*K.log(y_pred), axis=-1)
    loss1 = superclass1_loss(y_true, y_pred)
    return .5*loss0+.5*loss1

In [8]:
def struct2_loss(y_true,y_pred):
    # This is an equal blend of superclass2_loss, superclass1_loss 
    # and "normal" (class-level) loss
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    loss0 = K.sum(y_true*(-1)*K.log(y_pred), axis=-1)
    loss1 = superclass1_loss(y_true, y_pred)
    loss2 = superclass2_loss(y_true, y_pred)
    return (1/3)*loss0+(1/3)*loss1+(1/3)*loss2

In [9]:
def struct3_loss(y_true,y_pred):
    # This is an equal blend of superclass3_loss,superclass2_loss, 
    # superclass1_loss and "normal" (class-level) loss
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    loss0 = K.sum(y_true*(-1)*K.log(y_pred), axis=-1)
    loss1 = superclass1_loss(y_true, y_pred)
    loss2 = superclass2_loss(y_true, y_pred)
    loss3 = superclass3_loss(y_true, y_pred)
    return .25*loss0+.25*loss1+.25*loss2+.25*loss3

In [10]:
def super1_acc(y_true, y_pred):
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant([4, 30, 55, 72, 95, 1, 32, 67, 73, 91, 54, 62, 70, 82, 92, 9, 10, 16, 28, 61, 0, 51, 53, 57, 83, 
                          22, 39, 40, 86, 87, 5, 20, 25, 84, 94, 6, 7, 14, 18, 24, 3, 42, 43, 88, 97, 12, 17, 37, 68, 76, 
                          23, 33, 49, 60, 71, 15, 19, 21, 31, 38, 34, 63, 64, 66, 75, 26, 45, 77, 79, 99, 2, 11, 35, 46, 98,
                          27, 29, 44, 78, 93, 36, 50, 65, 74, 80, 47, 52, 56, 59, 96, 8, 13, 48, 58, 90, 41, 69, 81, 85, 89]),
        values=tf.constant([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 
                            7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 
                            13, 13, 13, 13, 13, 14, 14, 14, 14, 14,
                            15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19]),
    ),
    default_value=tf.constant(-1),
    name="superclass_index"
    )
    yt_hard = tf.argmax(y_true, axis=1)
    yp_hard = tf.argmax(y_pred, axis=1)
    
    ytc = table.lookup(tf.cast(yt_hard, tf.int32))
    ypc = table.lookup(tf.cast(yp_hard, tf.int32))
    return K.sum(tf.cast(tf.equal(ytc,ypc),tf.int32))/tf.shape(ytc)

In [11]:
def super2_acc(y_true, y_pred):
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant([4, 30, 55, 72, 95, 1, 32, 67, 73, 91, 54, 62, 70, 82, 92, 9, 10, 16, 28, 61, 0, 51, 53, 57, 83, 
                          22, 39, 40, 86, 87, 5, 20, 25, 84, 94, 6, 7, 14, 18, 24, 3, 42, 43, 88, 97, 12, 17, 37, 68, 76, 
                          23, 33, 49, 60, 71, 15, 19, 21, 31, 38, 34, 63, 64, 66, 75, 26, 45, 77, 79, 99, 2, 11, 35, 46, 98,
                          27, 29, 44, 78, 93, 36, 50, 65, 74, 80, 47, 52, 56, 59, 96, 8, 13, 48, 58, 90, 41, 69, 81, 85, 89]),
        values=tf.constant([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
                            2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
                            3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 
                            6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]),
    ),
    default_value=tf.constant(-1),
    name="superclass2_index"
    )
    yt_hard = tf.argmax(y_true, axis=1)
    yp_hard = tf.argmax(y_pred, axis=1)
    
    ytc = table.lookup(tf.cast(yt_hard, tf.int32))
    ypc = table.lookup(tf.cast(yp_hard, tf.int32))
    return K.sum(tf.cast(tf.equal(ytc,ypc),tf.int32))/tf.shape(ytc)

In [12]:
def super3_acc(y_true, y_pred):
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant([4, 30, 55, 72, 95, 1, 32, 67, 73, 91, 54, 62, 70, 82, 92, 9, 10, 16, 28, 61, 0, 51, 53, 57, 83, 
                          22, 39, 40, 86, 87, 5, 20, 25, 84, 94, 6, 7, 14, 18, 24, 3, 42, 43, 88, 97, 12, 17, 37, 68, 76, 
                          23, 33, 49, 60, 71, 15, 19, 21, 31, 38, 34, 63, 64, 66, 75, 26, 45, 77, 79, 99, 2, 11, 35, 46, 98,
                          27, 29, 44, 78, 93, 36, 50, 65, 74, 80, 47, 52, 56, 59, 96, 8, 13, 48, 58, 90, 41, 69, 81, 85, 89]),
        values=tf.constant([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                            1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 
                            3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
    ),
    default_value=tf.constant(-1),
    name="superclass2_index"
    )
    yt_hard = tf.argmax(y_true, axis=1)
    yp_hard = tf.argmax(y_pred, axis=1)
    
    ytc = table.lookup(tf.cast(yt_hard, tf.int32))
    ypc = table.lookup(tf.cast(yp_hard, tf.int32))
    return K.sum(tf.cast(tf.equal(ytc,ypc),tf.int32))/tf.shape(ytc)

## Begin Training loop

In [13]:
# Set some parameters
batch_size = 128
num_classes = 100
data_augmentation = True
train_set_size = 50000
num_trials = 1

timestamp = int(time.time())

In [14]:
def new_model(num_classes):


    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(96,96,3))

    x = base_model.output
    x_new  = GlobalAveragePooling2D()(x)
    x_new = Dense(num_classes, activation='softmax', name='fc10')(x_new)
    model = Model(inputs=base_model.input, outputs=x_new)

    # for layer in base_model.layers:
    #     layer.trainable = False

    return model

In [15]:
X_train = tf.keras.applications.resnet50.preprocess_input(X_train)
X_test = tf.keras.applications.resnet50.preprocess_input(X_test)

In [16]:
X_train.shape, X_test.shape

((50000, 96, 96, 3), (10000, 96, 96, 3))

In [17]:
y_train = tf.keras.utils.to_categorical(Y_train, num_classes)
y_test = tf.keras.utils.to_categorical(Y_test, num_classes)

In [18]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)


In [19]:
adam1 = tf.keras.optimizers.Adam(lr=.00001)
# sgd = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9, nesterov=True)

In [20]:
model = new_model(num_classes)
model.compile(loss='categorical_crossentropy',
              optimizer=adam1,
              metrics=['accuracy',super1_acc,super2_acc, super3_acc,
                       'categorical_crossentropy',#superclass1_loss, superclass2_loss,superclass3_loss,
                       #struct1_loss, struct2_loss, 
                       struct3_loss])

In [21]:
history=model.fit(datagen.flow(X_train, y_train,
                                 batch_size=batch_size),
                    epochs=250,
                    validation_data=(X_test, y_test),
                    workers=8, shuffle=True, verbose=1)


Epoch 1/250
391/391 [==============================] - 197s 468ms/step - loss: 4.7725 - accuracy: 0.0699 - super1_acc: 0.1510 - super2_acc: 0.2414 - super3_acc: 0.4199 - categorical_crossentropy: 4.7725 - struct3_loss: 2.6486 - val_loss: 2.7927 - val_accuracy: 0.3507 - val_super1_acc: 0.5149 - val_super2_acc: 0.5921 - val_super3_acc: 0.6723 - val_categorical_crossentropy: 2.7927 - val_struct3_loss: 1.5021
Epoch 2/250
391/391 [==============================] - 177s 451ms/step - loss: 2.4516 - accuracy: 0.4191 - super1_acc: 0.5783 - super2_acc: 0.6448 - super3_acc: 0.7061 - categorical_crossentropy: 2.4516 - struct3_loss: 1.3328 - val_loss: 1.7212 - val_accuracy: 0.5537 - val_super1_acc: 0.7042 - val_super2_acc: 0.7579 - val_super3_acc: 0.7952 - val_categorical_crossentropy: 1.7212 - val_struct3_loss: 0.8802
Epoch 3/250
391/391 [==============================] - 177s 450ms/step - loss: 1.5786 - accuracy: 0.5847 - super1_acc: 0.7279 - super2_acc: 0.7781 - super3_acc: 0.8106 - categorical_

In [22]:
model2 = new_model(num_classes)
model2.compile(loss=struct3_loss,
              optimizer=adam1,
              metrics=['accuracy',super1_acc,super2_acc, super3_acc,
                       'categorical_crossentropy',struct3_loss])

In [23]:
history2=model2.fit(datagen.flow(X_train, y_train,
                                 batch_size=batch_size),
                    epochs=250,
                    validation_data=(X_test, y_test),
                    workers=8, shuffle=True, verbose=1)


Epoch 1/250
391/391 [==============================] - 196s 468ms/step - loss: 1.6711 - accuracy: 0.2555 - super1_acc: 0.4657 - super2_acc: 0.5535 - super3_acc: 0.6344 - categorical_crossentropy: 3.3561 - struct3_loss: 1.6711 - val_loss: 0.6474 - val_accuracy: 0.6059 - val_super1_acc: 0.7752 - val_super2_acc: 0.8177 - val_super3_acc: 0.8416 - val_categorical_crossentropy: 1.3814 - val_struct3_loss: 0.6474
Epoch 2/250
391/391 [==============================] - 179s 457ms/step - loss: 0.5842 - accuracy: 0.6388 - super1_acc: 0.8008 - super2_acc: 0.8436 - super3_acc: 0.8614 - categorical_crossentropy: 1.2547 - struct3_loss: 0.5842 - val_loss: 0.5235 - val_accuracy: 0.6746 - val_super1_acc: 0.8213 - val_super2_acc: 0.8596 - val_super3_acc: 0.8774 - val_categorical_crossentropy: 1.1164 - val_struct3_loss: 0.5235
Epoch 3/250
391/391 [==============================] - 179s 457ms/step - loss: 0.4528 - accuracy: 0.7088 - super1_acc: 0.8472 - super2_acc: 0.8790 - super3_acc: 0.8939 - categorical_

In [24]:
str_name_1 = 'resnet_imagenet_250_1e5_adam_std'
str_name_2 = 'resnet_imagenet_250_1e5_adam_struc'


In [25]:
import json

In [26]:
with open(str_name_1, 'w') as outfile:
    json.dump(history.history, outfile)

In [27]:
with open(str_name_2, 'w') as outfile:
    json.dump(history2.history, outfile)